In [ ]:
import mysql.connector
from sentence_transformers import SentenceTransformer

In [ ]:
SEARCH_KEYWORD = 'サンプル'
# SEARCH_KEYWORD = 'これはテスト文です。'

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
model

In [ ]:
search_keyword_embedding = model.encode([SEARCH_KEYWORD])[0]
search_keyword_embedding_str = ','.join(map(str, search_keyword_embedding))
search_keyword_embedding

In [ ]:
import os
db_config = {
    'user': os.environ['MYSQL_USER'],
    'password': os.environ['MYSQL_PASSWORD'],
    'host': 'llm-rag-examples-mysql',
    'database': os.environ['MYSQL_DATABASE'],
}

db_config

In [ ]:
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

In [ ]:
cursor.execute('SELECT text, embedding FROM text_embeddings')

In [ ]:
import numpy as np
search_result: list = []
for (text, embedding,) in cursor:
    print(embedding)
    print(type(embedding))
    dot_product = np.dot(np.array(embedding), search_keyword_embedding)
    search_result.append({
        'text': text,
        'embedding': embedding,
        'dot_product': dot_product,
    })

In [ ]:
cursor.close()
conn.close()

In [ ]:
search_result

In [ ]:
sorted_result_full = sorted(search_result, key=lambda x: x['dot_product'], reverse=True)

In [ ]:
sorted_result = [(l.get('text'), l.get('dot_product')) for l in sorted_result_full]